In [ ]:
import datetime
import cv2
from ultralytics import YOLO

# function to return IoU
def intersection_over_union(boxA,boxB):
    # xmin, ymin, xmax, ymax of intersection box
    x_min = max(boxA[0],boxB[0])
    y_min = max(boxA[1],boxB[1])
    x_max = min(boxA[2],boxB[2])
    y_max = min(boxA[3],boxB[3])

    # Intersection 영역
    intersection_area = max(x_max-x_min,0) * max(y_max-y_min,0)
    # boxA, boxB 영역
    boxA_area = (boxA[2]-boxA[0])*(boxA[3]-boxA[1])
    boxB_area = (boxB[2]-boxB[0])*(boxB[3]-boxB[1])
    # Total 영역
    total_area = boxA_area + boxB_area - intersection_area
    IoU = intersection_area/total_area
    return IoU

coco128 = open('./coco128.txt', 'r')
data = coco128.read()
class_list = data.split('\n')
coco128.close()

# 최소 정확도 이하의 객체는 화면에 출력하지 않음
CONFIDENCE_THRESHOLD = 0.8
GREEN = (0, 255, 0)       # 객체 box는 녹색으로
WHITE = (255, 255, 255)   # 객체 name은 흰색으로
BLUE = (0, 0, 255)        # FPS 추적은 청색으로

model = YOLO('./yolov8s.pt')

# 웹캠 setting
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Initialize variables for object tracking
frame_counter = 0
detection_frequency = 1
prev_boxes = []
threshold_iou = 0.8

while True:
    start_time = datetime.datetime.now()
    # fps를 계산하기 위해 동작 시작 시간을 저장
    success, frame = cap.read()
    if not success:
        print('Cam Error')
        break
    
    frame_counter += 1

    if frame_counter % detection_frequency == 0:
        # read a frame from the video
        detection = model.predict(frame,
                              conf=CONFIDENCE_THRESHOLD)[0]
        current_boxes = []
        for data in detection.boxes.data.tolist(): 
            # data : [xmin, ymin, xmax, ymax, confidence_score, class_id]
            xmin, ymin, xmax, ymax, conf, label = int(data[0]), int(data[1]), int(data[2]), int(data[3]), float(data[4]), int(data[5])
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), GREEN, 2)
            cv2.putText(frame, class_list[label]+' '+str(round(conf, 2)), (xmin, ymin-10), cv2.FONT_ITALIC, 1, WHITE, 2)

            current_boxes.append((xmin, ymin, xmax, ymax))

            if len(prev_boxes) < len(current_boxes):
                # new object detected
                object_name = class_list[label]
                print("New object detected:", object_name)
            else:
                # check for significant movement
                iou_btw_frame = [intersection_over_union(box, (xmin,ymin,xmax,ymax))for box in prev_boxes]
                if max(iou_btw_frame) < threshold_iou:
                    object_name = class_list[label]
                    print("Object moved significantly:", object_name)

        prev_boxes = current_boxes
        
        end_time = datetime.datetime.now()
        total = (end_time - start_time).total_seconds()
        print(f'Time to process 1 frame: {total * 1000:.0f} milliseconds')

        fps = f'FPS: {1 / total:.2f}'
        print(fps)
        print(frame_counter)

        cv2.putText(frame, fps, (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, BLUE, 2)
        cv2.imshow('frame', frame)

        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()